# spark dataframe

In [1]:
from pyspark import SparkContext, SparkConf
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
import pandas as pd 

In [2]:
from pyspark.sql.functions import concat, col, lit

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
path = '/data/lsml/sga/clickstream.csv'
df = spark.read.option("delimiter", "\t").option("header", "true").csv(path)

In [5]:
df.show(10)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|       event|      main|1620494801|
|    562|       507|wNaxLlerrorU|      main|1620494808|
|    562|       507|       event|      main|1620494808|
|    562|       507|       event|      main|1620494808|
|    562|       507|       event|      main|1620494814|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
+-------+----------+------------+----------+----------+
only showing top 10 rows



In [6]:
from pyspark.sql.functions import lower

In [7]:
df2 = df.withColumn('event_type', lower(col('event_type')))

In [8]:
df2.show(5)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|       event|      main|1620494801|
|    562|       507|wnaxllerroru|      main|1620494808|
+-------+----------+------------+----------+----------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import lower, col, concat, lit, concat_ws
from pyspark.sql import functions as F

In [10]:
df3 = df2.filter(col("event_type").like("%error%"))

In [11]:
df4 = df3.groupBy("user_id", "session_id").agg(F.min(col("timestamp")))

In [12]:
df4 = df4.withColumnRenamed("user_id", "uid").withColumnRenamed("session_id", "ssid")
df4.show(4)

+---+----+--------------+
|uid|ssid|min(timestamp)|
+---+----+--------------+
| 10|2151|    1645007103|
|101|1121|    1623323672|
|101|1407|    1628754322|
|105|1341|    1643382743|
+---+----+--------------+
only showing top 4 rows



In [13]:
cond = [df2.user_id == df4.uid, df2.session_id == df4.ssid]
df5 = df2.join(df4, cond, how='left').drop(df4.uid).drop(df4.ssid)

In [14]:
df5.show()

+-------+----------+----------+----------+----------+--------------+
|user_id|session_id|event_type|event_page| timestamp|min(timestamp)|
+-------+----------+----------+----------+----------+--------------+
|      0|       882|      page|      main|1628200239|          null|
|      0|       882|     event|      main|1628200525|          null|
|      0|       882|     event|      main|1628201547|          null|
|      0|       882|      page|   tariffs|1628201756|          null|
|      0|       882|     event|   tariffs|1628201860|          null|
|      0|       882|      page|   digital|1628201910|          null|
|      0|       882|      page|      main|1628231061|          null|
|      0|       882|     event|      main|1628231147|          null|
|      0|       882|      page|      news|1628231334|          null|
|      0|       882|      page|      main|1628244073|          null|
|      0|       882|     event|      main|1628245072|          null|
|      0|       882|     event|   

In [15]:
cond2 = (col("timestamp") < col("min(timestamp)")) | (col("min(timestamp)").isNull())
df6 = df5.filter(cond2)

In [16]:
df7 = df6.filter(col("event_type") == "page")

In [17]:
df7.show()

+-------+----------+----------+----------+----------+--------------+
|user_id|session_id|event_type|event_page| timestamp|min(timestamp)|
+-------+----------+----------+----------+----------+--------------+
|      0|       882|      page|      main|1628200239|          null|
|      0|       882|      page|   tariffs|1628201756|          null|
|      0|       882|      page|   digital|1628201910|          null|
|      0|       882|      page|      main|1628231061|          null|
|      0|       882|      page|      news|1628231334|          null|
|      0|       882|      page|      main|1628244073|          null|
|      0|       882|      page|   tariffs|1628247321|          null|
|      0|       882|      page|   digital|1628248028|          null|
|      0|       882|      page|   archive|1628248153|          null|
|      0|       882|      page|      main|1628249336|          null|
|      1|      1135|      page|      main|1628725420|          null|
|      1|      1135|      page|   

In [18]:
df8 = df7.groupBy("user_id", "session_id").agg(F.collect_list("event_page"))

In [19]:
df8.show()

+-------+----------+------------------------+
|user_id|session_id|collect_list(event_page)|
+-------+----------+------------------------+
|      0|       882|    [main, tariffs, d...|
|      1|      1135|         [main, archive]|
|     10|      1224|     [main, news, bonus]|
|     10|      1501|         [main, digital]|
|     10|      2151|    [main, digital, t...|
|    100|      2006|          [main, family]|
|    101|      1121|    [main, family, ar...|
|    101|      1407|    [main, family, ar...|
|    103|       552|          [main, family]|
|    104|      1343|    [main, news, bonu...|
|    104|       616|    [main, tariffs, m...|
|    105|      1341|    [main, archive, f...|
|    105|       715|                  [main]|
|    106|       916|                  [main]|
|    107|       520|    [main, digital, n...|
|    108|       329|    [main, digital, t...|
|     11|      1598|                  [main]|
|     11|      2114|                  [main]|
|    112|      1173|    [main, dig

In [20]:
df9 = df8.groupby("collect_list(event_page)").count()

In [21]:
df9.show()

+------------------------+-----+
|collect_list(event_page)|count|
+------------------------+-----+
|           [main, bonus]| 3491|
|    [main, archive, d...|    1|
|    [main, news, arch...|    3|
|    [main, archive, f...|   16|
|    [main, family, di...|    1|
|    [main, digital, b...|    1|
|    [main, news, main...|    1|
|    [main, news, digi...|    1|
|    [main, archive, t...|    1|
|    [main, tariffs, n...|    1|
|    [main, archive, d...|    1|
|    [main, tariffs, f...|    1|
|    [main, family, ar...|    1|
|    [main, tariffs, a...|    1|
|    [main, family, ne...|    1|
|    [main, tariffs, b...|    1|
|    [main, bonus, new...|    1|
|    [main, bonus, dig...|    1|
|    [main, family, ma...|    1|
|    [main, digital, b...|    1|
+------------------------+-----+
only showing top 20 rows



In [22]:
df9.sort(col("count").desc()).show(30)

+------------------------+-----+
|collect_list(event_page)|count|
+------------------------+-----+
|                  [main]|39256|
|         [main, tariffs]| 6523|
|            [main, news]| 6260|
|         [main, archive]| 5834|
|          [main, family]| 4854|
|         [main, digital]| 4212|
|           [main, bonus]| 3485|
|    [main, tariffs, n...| 1184|
|    [main, news, tari...| 1131|
|    [main, tariffs, a...| 1032|
|    [main, news, arch...|  995|
|    [main, archive, n...|  993|
|    [main, archive, t...|  989|
|    [main, family, ta...|  919|
|    [main, news, family]|  913|
|    [main, tariffs, f...|  910|
|    [main, family, news]|  877|
|    [main, archive, f...|  816|
|    [main, news, digi...|  794|
|    [main, family, ar...|  769|
|    [main, tariffs, m...|  760|
|    [main, tariffs, d...|  748|
|    [main, digital, n...|  744|
|    [main, digital, t...|  720|
|    [main, archive, d...|  718|
|         [main, spravka]|  707|
|      [main, news, main]|  681|
|    [main

In [25]:
df9 = df9.orderBy('count', ascending=False)

In [23]:
df9.show(30)

+------------------------+-----+
|collect_list(event_page)|count|
+------------------------+-----+
|                  [main]|39256|
|         [main, tariffs]| 6519|
|            [main, news]| 6257|
|         [main, archive]| 5835|
|          [main, family]| 4854|
|         [main, digital]| 4212|
|           [main, bonus]| 3486|
|    [main, tariffs, n...| 1185|
|    [main, news, tari...| 1129|
|    [main, tariffs, a...| 1032|
|    [main, news, arch...|  996|
|    [main, archive, n...|  994|
|    [main, archive, t...|  990|
|    [main, family, ta...|  916|
|    [main, news, family]|  912|
|    [main, tariffs, f...|  911|
|    [main, family, news]|  877|
|    [main, archive, f...|  815|
|    [main, news, digi...|  793|
|    [main, family, ar...|  770|
|    [main, tariffs, m...|  763|
|    [main, tariffs, d...|  749|
|    [main, digital, n...|  743|
|    [main, digital, t...|  719|
|    [main, archive, d...|  718|
|         [main, spravka]|  705|
|      [main, news, main]|  683|
|    [main

In [26]:
ans3 = df9.take(30)

In [29]:
string = ''
for content in ans3:
    string += '%s\t%s\n' % (content[0], content[1])

In [33]:
print(string)

[u'main']	39256
[u'main', u'tariffs']	6522
[u'main', u'news']	6263
[u'main', u'archive']	5832
[u'main', u'family']	4852
[u'main', u'digital']	4214
[u'main', u'bonus']	3484
[u'main', u'tariffs', u'news']	1186
[u'main', u'news', u'tariffs']	1129
[u'main', u'tariffs', u'archive']	1032
[u'main', u'news', u'archive']	998
[u'main', u'archive', u'news']	993
[u'main', u'archive', u'tariffs']	991
[u'main', u'family', u'tariffs']	919
[u'main', u'news', u'family']	913
[u'main', u'tariffs', u'family']	911
[u'main', u'family', u'news']	878
[u'main', u'archive', u'family']	817
[u'main', u'news', u'digital']	793
[u'main', u'family', u'archive']	769
[u'main', u'tariffs', u'main']	759
[u'main', u'tariffs', u'digital']	747
[u'main', u'digital', u'news']	745
[u'main', u'digital', u'tariffs']	720
[u'main', u'archive', u'digital']	718
[u'main', u'spravka']	705
[u'main', u'news', u'main']	684
[u'main', u'digital', u'archive']	680
[u'main', u'tariffs', u'bonus']	666
[u'main', u'archive', u'main']	622



In [34]:
with open('week6_ans3_sang.tsv', 'w') as f:
    f.write(string)